In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

In [3]:
df = pd.read_csv('Real estate.csv')
df.head()

,No,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude,Y house price of unit area
0,1,2012.917,32.0,84.87882,10,24.98298,121.54024,37.9
1,2,2012.917,19.5,306.59470,9,24.98034,121.53951,42.2
2,3,2013.583,13.3,561.98450,5,24.98746,121.54391,47.3
3,4,2013.500,13.3,561.98450,5,24.98746,121.54391,54.8
4,5,2012.833,5.0,390.56840,5,24.97937,121.54245,43.1


In [4]:
inputData = ""
outputData = ""

def initialiseData():
    global outputData
    global inputData
    outputData = df.iloc[:,7]
    inputData = df.iloc[:,1:7]

def meanData(inputData):
    return np.mean(inputData)

def sdData(inputData):
    return np.std(inputData)

initialiseData()

In [5]:
numColumns = inputData.shape[1]
startingWeight = np.zeros(numColumns+1)

In [6]:
def normaliseData(inputData):
    meanInput = meanData(inputData)
    stdInput = sdData(inputData)
    return ((inputData-meanInput)/stdInput)

In [7]:
def summation(inputData,outputData,numRows):
    sum = 0
    i=0
    while i < numRows:
        sum = sum + (outputData.iloc[i] - np.dot(inputData.iloc[i],startingWeight))*inputData.iloc[i]
        i = i+1
    return sum
    
def insertCol(inputData):
    value = pd.DataFrame(np.ones(inputData.shape[0]))
    inputData.insert(0,"Bias weight",value)

def gradientDescent(iterations,learningRate,startingWeight,inputData,outputData,numColumns):
    insertCol(inputData)
    numRows = inputData.shape[0]
    for i in range(iterations):
        tempWeight = np.zeros(numColumns+1)
        tempWeight = startingWeight + learningRate*(summation(inputData,outputData,numRows)/numRows)
        startingWeight = tempWeight
    return startingWeight

def loss(rowsPerFold,newWeight,testInput,trainOutputStd,trainOutputMean,testOutput):
        currLoss=0
        j=0
        while j< rowsPerFold:
            predictedNormalised = newWeight[0]+np.dot(newWeight[1:],testInput.iloc[j])
            predicted = predictedNormalised*trainOutputStd+trainOutputMean
            currLoss += (predicted-testOutput.iloc[j])**2
            j = j+1
        currLoss = currLoss/rowsPerFold
        currLoss = np.sqrt(currLoss)
        return currLoss
    
    

In [8]:
def kFold(k,iterations,learningRate,startingWeight):
    numColumns = inputData.shape[1]
    totalRows = inputData.shape[0]
    rowsPerFold = int(totalRows/k)
    averageLoss = 0
    i=0
    while i<k:
        start = i*rowsPerFold
        end = start+rowsPerFold
        testInput = inputData[start:end]
        testOutput = outputData[start:end]
        trainInput = inputData.drop(inputData.index[start:end])
        trainOutput = outputData.drop(outputData.index[start:end])
        trainInputMean = meanData(trainInput)
        trainInputStd = sdData(trainInput)
        trainOutputMean = meanData(trainOutput)
        trainOutputStd = sdData(trainOutput)
        trainInput = normaliseData(trainInput)
        trainOutput = normaliseData(trainOutput)

        startingWeight = np.zeros(numColumns+1)
        newWeight = gradientDescent(iterations,learningRate,startingWeight,trainInput,trainOutput,numColumns)
        testInput = (testInput-trainInputMean)/trainInputStd
        averageLoss += loss(rowsPerFold,newWeight,testInput,trainOutputStd,trainOutputMean,testOutput)
        i = i+1
    averageLoss = averageLoss/k
    return averageLoss




In [9]:
def runGD(k,learningRate,iterations):
    print("Learning Rate: ",learningRate," Iterations: ",iterations," k: ",k)
    averageLoss = kFold(k,iterations,learningRate,startingWeight.copy())
    print("Average Loss: ",averageLoss)

In [13]:
for i in range(2,6):
    runGD(i,0.0005,500)

Learning Rate:  0.0005  Iterations:  500  k:  2


c:\Users\Vibhor\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
c:\Users\Vibhor\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


Average Loss:  9.8765111671209
Learning Rate:  0.0005  Iterations:  500  k:  3


c:\Users\Vibhor\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
c:\Users\Vibhor\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
c:\Users\Vibhor\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


Average Loss:  9.858503749298007
Learning Rate:  0.0005  Iterations:  500  k:  4


c:\Users\Vibhor\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
c:\Users\Vibhor\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
c:\Users\Vibhor\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
c:\Users\Vibhor\anaconda3\li

Average Loss:  9.793230561567986
Learning Rate:  0.0005  Iterations:  500  k:  5


c:\Users\Vibhor\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
c:\Users\Vibhor\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
c:\Users\Vibhor\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
c:\Users\Vibhor\anaconda3\li

Average Loss:  9.757021173896524
